# __텍스트로 텍스트 검색하기__

- 튜토리얼 난이도 : ★☆☆☆☆
- 읽는데 걸리는 시간 : 7분
- 사용 언어 : [SQL](https://ko.wikipedia.org/wiki/SQL) (100%)
- 실행 파일 위치 : tutorial/thanosql_search/search_text_by_text.ipynb   
- 참고 문서 : [단어 임베딩: 어휘의 의미(LEXICAL SEMANTICS)를 인코딩하기](https://tutorials.pytorch.kr/beginner/nlp/word_embeddings_tutorial.html), [컴퓨터가 바라보는 문자](https://heung-bae-lee.github.io/2020/01/16/NLP_01/)

## 튜토리얼 소개

<div class="admonition note">
    <h4 class="admonition-title">텍스트 수치화 기술 이해하기</h4>
    <p>컴퓨터는 사람이 쓰는 언어(자연어)를 곧바로 입력으로 받을 수 없습니다. 때문에 자연어를 기계가 인식할 수 있는 수치 데이터로 변환하는 과정이 필요합니다. 자연어 처리분야에서 임베딩이란 이렇게 사람이 쓰는 자연어를 기계가 이해할 수 있는 숫자형태인 vector로 바꾼 결과 혹은 그 일련의 과정 전체를 의미합니다.</p>
</div>

자연어의 임베딩을 구하는 기법은 크게 통계적인 기법과, 인공신경망 기반 기법으로 구분되고, 세부적으로도 더 나눌 수 있습니다. ThanoSQL에서는 인공신경망 기반의 자가학습모델을 사용하여, 텍스트를 잘 표현하는 벡터를 스스로 학습하는 방법을 제공합니다.

<div class="admonition note">
    <h4 class="admonition-title">본 튜토리얼에서는</h4>
    <p>👉 이번 튜토리얼에서는 <mark style="background-color:#FFD79C">영화 리뷰 데이터</mark>를 사용합니다. 데이터는 영화 리뷰 텍스트와, 라벨값으로 구성되어있지만, 자가학습법을 사용해 훈련하는 예제를 보여드리기 위해 라벨값은 사용하지 않습니다. 10000개의 영화 리뷰 데이터를 학습하여, 입력 문장과 유사한 문장 찾기, 문장의 키워드 추출하기를 해보겠습니다.</p>
</div>

## __0. 데이터 세트 준비__

ThanoSQL의 쿼리 구문을 사용하기 위해서는 [ThanoSQL 워크스페이스](https://docs.thanosql.ai/getting_started/how_to_use_ThanoSQL/#5-thanosql)
에서 언급된 것처럼 API 토큰을 생성하고 아래의 쿼리를 실행해야 합니다.

In [ ]:
%load_ext thanosql
%thanosql API_TOKEN=<발급받은_API_TOKEN>

### __데이터 세트 준비__

In [3]:
%%thanosql
GET THANOSQL DATASET nsmc_data
OPTIONS (overwrite=True)

Success


<div class="admonition note">
    <h4 class="admonition-title">쿼리 세부 정보</h4>
    <ul>
        <li>"<strong>GET THANOSQL DATASET</strong>" 쿼리 구문을 사용하여 원하는 데이터 세트를 워크스페이스에 저장합니다. </li>
        <li>"<strong>OPTIONS</strong>" 쿼리 구문을 통해 <strong>GET THANOSQL DATASET</strong> 에 사용할 옵션을 지정합니다.
        <ul>
            <li>"overwrite" : 동일 이름의 데이터 세트가 존재하는 경우 덮어쓰기 가능 유무 설정. True일 경우 기존 데이터 세트는 새로운 데이터 세트로 변경됨 (True|False, DEFAULT : False) </li>
        </ul>
        </li>
    </ul>
</div>

In [4]:
%%thanosql
COPY nsmc_train
OPTIONS (overwrite=True)
FROM "thanosql-dataset/nsmc_data/nsmc_sample_train.csv"

Success


In [5]:
%%thanosql
COPY nsmc_test
OPTIONS (overwrite=True)
FROM "thanosql-dataset/nsmc_data/nsmc_sample_test.csv"

Success


<div class="admonition note">
    <h4 class="admonition-title">쿼리 세부 정보</h4>
    <ul>
        <li>"<strong>COPY</strong>" 쿼리 구문을 사용하여 DB에 저장 할 데이터 세트명을 지정합니다. </li>
        <li>"<strong>OPTIONS</strong>" 쿼리 구문을 통해 <strong>COPY</strong> 에 사용할 옵션을 지정합니다.
        <ul>
            <li>"overwrite" : 동일 이름의 데이터 세트가 DB상에 존재하는 경우 덮어쓰기 가능 유무 설정. True일 경우 기존 데이터 세트는 새로운 데이터 세트로 변경됨 (True|False, DEFAULT : False) </li>
        </ul>
        </li>
    </ul>
</div>

## __1. 데이터 세트 확인__

텍스트 수치화 모델을 만들기 위해 ThanoSQL DB에 저장되어 있는 <mark style="background-color:#FFEC92">nsmc_train</mark> 테이블을 사용합니다. <mark style="background-color:#FFEC92">nsmc_train</mark> 테이블은 <mark style="background-color:#FFD79C">NAVER Sentiment Movie Corpus</mark> 영화 리뷰 데이터 및 라벨 정보의 일부가 담겨 있는 테이블입니다. 아래의 쿼리문을 실행하고 테이블의 내용을 확인합니다.

In [6]:
%%thanosql
SELECT *
FROM nsmc_train
LIMIT 5

,id,document,label
0,9465891,저퀄리티. 뭐하나 장점이 없음.,0
1,9792014,10점 만점에 1점 !!! 최악이네 하아...,0
2,3795348,누가 억지눈물 잘흘리나 자기네들끼리 대결하는 작품,0
3,1129393,신도 싫고 인간도 싫다,1
4,4678610,추억의 명작만화,1


<div class="admonition note">
    <h4 class="admonition-title">데이터 테이블 이해하기</h4>
    <p><mark style="background-color:#FFEC92">nsmc_train</mark> 테이블은 아래와 같은 정보를 담고 있습니다.</p>
    <ul>
        <li><mark style="background-color:#D7D0FF">id</mark>: 데이터의 식별 번호</li>
        <li><mark style="background-color:#D7D0FF">document</mark>: 영화 리뷰 데이터</li>
        <li><mark style="background-color:#D7D0FF">label</mark> : 라벨값</li>
    </ul>
</div>

## __2. 텍스트 수치화 모델 생성__

이전 단계에서 확인한 <mark style="background-color:#FFEC92">nsmc_train</mark> 테이블을 사용하여 텍스트 수치화 모델을 만듭니다. 아래의 쿼리 구문을 실행하여 <mark style="background-color:#E9D7FD">nsmc_text_search_model</mark>이라는 이름의 모델을 만듭니다.  
(쿼리 실행 시 예상 소요 시간 : 2 min)

In [7]:
%%thanosql
BUILD MODEL nsmc_text_search_model
USING SBERTKo
OPTIONS (
    text_col="document",
    overwrite=True
)
AS
SELECT *
FROM nsmc_train

Building model...
Success


<div class="admonition note">
    <h4 class="admonition-title">쿼리 세부 정보</h4>
    <ul>
        <li>"<strong>BUILD MODEL</strong>" 쿼리 구문을 사용하여 <mark style="background-color:#E9D7FD">nsmc_text_search_model</mark> 이라는 모델을 만들고 학습시킵니다.</li>
        <li>"<strong>USING</strong>" 쿼리 구문을 통해 베이스 모델로 <mark style="background-color:#E9D7FD">SBERTKo</mark> 모델을 사용할 것을 명시합니다.</li>
        <li>"<strong>OPTIONS</strong>" 쿼리 구문을 통해 모델 생성에 사용할 옵션을 지정합니다.
        <ul>
            <li>"text_col" : 데이터 테이블에서 영화 리뷰 데이터를 담은 컬럼</li>
            <li>"epochs" : 텍스트 수치화 모델을 생성하기 위한 데이터 세트 학습 횟수 (int, DEFAULT : 1)</li>
            <li>"batch_size" : 한 번의 예측에서 읽는 데이터 세트 묶음의 크기 (int, DEFAULT : 16)</li> 
            <li>"learning_rate" : 모델의 학습률 (float, DEFAULT : 3e-5)</li> 
            <li>"train" : False일 경우 사전훈련된 모델을 추가로 학습하지 않고 그대로 사용 (True|False, DEFAULT : True)</li> 
            <li>"overwrite" : 동일 이름의 모델이 존재하는 경우 덮어쓰기 가능 유무 설정. True일 경우 기존 모델은 새로운 모델로 변경됨 (True|False, DEFAULT : False)</li>
        </ul>
        </li>
    </ul>
</div>

다음 "__CONVERT USING__ " 쿼리 구문을 실행하여 `nsmc_test` 텍스트 데이터들을 수치화 합니다. 수치화 된 결과는 `nsmc_test` 테이블에 <mark style="background-color:#D7D0FF ">nsmc_text_search_model_sbertko</mark>이라는 새로운 이름의 컬럼에 저장됩니다.

In [8]:
%%thanosql
CONVERT USING nsmc_text_search_model
OPTIONS (
    text_col="document",
    table_name="nsmc_test",
    batch_size=32
    )
AS 
SELECT *
FROM nsmc_test

,id,document,label,convert_result
0,9181084,꼭 한번 봐야하는 영화인 것 같네요 ㅎㅎ,1,"[-0.021847637, -0.008228128, -0.07573159, -0.0..."
1,6318649,권선징악은안드로메다로~럽라인은작가빙의된니끼한조연과~여주는남자갖고논질나쁜여자,0,"[-0.040243365, -0.016085487, -0.011301089, -0...."
2,5718332,유쾌하고 신나는 스피드 코미디.,1,"[-0.057554703, -0.044752803, -0.011217645, -0...."
3,8063675,정말 따뜻했던 드라마ㅠㅠ,1,"[-0.012758432, -0.025050018, -0.033760767, -0...."
4,10229366,솔직히 배우들의 연기는 나름 괜찮았다 하지만 냉정하게 영화자체만을 놓고보면 별다른 ...,0,"[-0.01959051, -0.0147701055, 0.035844997, -0.0..."
...,...,...,...,...
1995,5382227,30년간 본 수천편의 드라마중 최고는 이 작품,1,"[-0.039317492, -0.016572807, -0.025051735, -0...."
1996,4584773,빛돌을 이길 5명의 용사들의 패배,1,"[-0.057277907, 0.0052375863, -0.0078345, 0.012..."
1997,2559484,빌머레이의 굳은 표정에 모든것이 살아있다.. 짐자무시 최고!,1,"[-0.031670548, -0.0037371023, -0.06549691, 0.0..."
1998,9867081,이런영화에 투자하지 않는것이 진정한 한국영화의 발전을위한 초석,0,"[-0.0103046475, 0.028476555, -0.009558885, 0.0..."


<div class="admonition note">
    <h4 class="admonition-title">쿼리 세부 정보</h4>
    <ul>
        <li>"<strong>CONVERT USING</strong>" 쿼리 구문은 <code>nsmc_text_search_model</code>을 텍스트 수치화를 위한 알고리즘으로 사용합니다.</li>
        <li>"<strong>OPTIONS</strong>" 쿼리 구문을 통해 텍스트 수치화 시 필요한 변수들을 정의합니다.
        <ul>
            <li>"text_col" : 데이터 테이블에서 영화 리뷰 데이터를 담은 컬럼</li>
            <li>"table_name" : ThanoSQL DB 내에 저장될 테이블 이름을 정의합니다.</li>
            <li>"batch_size" : 한 번의 예측에서 읽는 데이터 세트 묶음의 크기</li> 
        </ul>
        </li>
    </ul>
</div>

## __3. 텍스트 수치화 모델을 사용해서 유사한 문서 검색하기__

이번 단계에서는 <mark style="background-color:#E9D7FD">nsmc_text_search_model</mark> 텍스트 수치화 모델과 테스트 테이블을 사용하여 유사한 문서를 검색합니다.

In [11]:
%%thanosql
SELECT document, label, search_result as score
FROM (
    SEARCH TEXT text="영화 내용이 불만족스러웠다"
    USING nsmc_text_search_model
    OPTIONS(emb_col = 'convert_result')
    AS 
    SELECT * 
    FROM nsmc_test
    )
ORDER BY score DESC 
LIMIT 10

Searching...


,document,label,score
0,"""진부한영화 강북고 서한길 남김..""""""",0,0.710047
1,"""전체관람가가 아니라 """"미취학아동전용""""이네요""",0,0.691292
2,"""""""세상은 친절한 곳이 아니에요""""""",1,0.651272
3,"""""""다녀왔습니다""""가 제일 무서웠음.""",0,0.644403
4,"""""""관심받고 싶었습니다. 여러분을 낚은거 죄송"""" 맞제?""",0,0.626293
5,시나리오는 한없이 후진데다가 썩은 동선이 가득한 영화,0,0.608278
6,"아이 눈높이에 맞는 영화, 어른은 다소 지루함.",1,0.607945
7,왕건 후속편으로써 매우 실망스런 작품.,0,0.596401
8,"실화에 충실한 영화라 잼없네요..소재 식상, 배경음으로 섞인듯한 인상, 흥미 없음",0,0.588617
9,내가 생각했던 스토리와 조금 달랐지만 신선하고 독특했던 영화였다.,1,0.587804


In [13]:
%%thanosql
SELECT document, label, search_result as score
FROM (
    SEARCH TEXT text="기분이 좋아지는 작품"
    USING nsmc_text_search_model
    OPTIONS(emb_col = 'convert_result')
    AS 
    SELECT * 
    FROM nsmc_test
    )
ORDER BY score DESC 
LIMIT 10

Searching...


,document,label,score
0,"""""""다녀왔습니다""""가 제일 무서웠음.""",0,0.723232
1,"""""""세상은 친절한 곳이 아니에요""""""",1,0.711677
2,"""""""관심받고 싶었습니다. 여러분을 낚은거 죄송"""" 맞제?""",0,0.707257
3,"""진부한영화 강북고 서한길 남김..""""""",0,0.675729
4,"""대박이다 이시대 최고의 영화! """"뭐야, 론머맨이잖아! 론머맨~~치지지지""""""",1,0.673560
5,"""전체관람가가 아니라 """"미취학아동전용""""이네요""",0,0.669445
6,정말 좋은영화네요'!,1,0.659650
7,인상 깊은 영화다~,1,0.647705
8,"자극적이지 않고, 따뜻한 감동을 선사하는 최고의 드라마!",1,0.618299
9,보고나면 기분 좋은 영화,1,0.617234


<div class="admonition note">
    <h4 class="admonition-title">쿼리 세부 정보</h4>
    <ul>
        <li>"<strong>SEARCH TEXT [images|audio|videos|texts|keywords]</strong>" 쿼리 구문은 검색하고자 하는 이미지|오디오|비디오|텍스트|키워드 데이터를 정의합니다.</li>
        <li>"<strong>USING</strong>"은 텍스트 수치화에 사용할 모델을 정의합니다.</li>
        <li>"<strong>AS</strong>" 쿼리 구문은 검색에 사용할 임베딩 테이블을 정의합니다. <code>nsmc_test</code> 테이블을 사용합니다.</li>
    </ul>
</div>

## __4. 텍스트 수치화 모델을 사용해서 문서에서 키워드 추출하기__

이번 단계에서는 <mark style="background-color:#E9D7FD">nsmc_text_search_model</mark> 텍스트 수치화 모델과 테스트 테이블을 사용하여 유사한 문서에서 키워드 추출합니다. 

In [14]:
%%thanosql
SEARCH KEYWORD
USING nsmc_text_search_model
OPTIONS (
    text_col="document",
    ngram_range=[1, 3],
    use_stopwords=True
    )
AS 
SELECT * 
FROM nsmc_test
LIMIT 10 OFFSET 40

Searching...


,id,document,label,convert_result,keyword
0,7928591,"일단 엠씨 역량 부족이 가장 큰 문제인듯. 시간 없다고, 재미없다고 게스트 말 끊는...",0,"[-0.01636468, 0.00073600485, -0.037863642, -0....","{'keyword': ['라고 끝 엠씨가노답임', '한마디 라고 끝', '시간 재미..."
1,8380896,방금 슬쩍 봤는데 '인삼' 처럼 생긴 '산삼' 암수가 서로 토닥이다가 주인공에게 잡...,0,"[-0.057706382, 0.0056580813, -0.06902707, -0.0...","{'keyword': ['는데 인삼 처럼', '주인공 잡히 ᆷ', '서로 토닥이 다..."
2,8914833,신세계에 비교하는 거 진심 빡친다. 신성모독,1,"[-0.02587987, 0.007081825, -0.039368443, 0.042...","{'keyword': ['빡 치 신성모독', '비교 거 진심', '진심 빡 치', ..."
3,9702804,킬링타임 최고다 열자채워,1,"[0.024242267, -0.039235096, -0.07083085, 0.015...","{'keyword': ['킬링타임 최고 다', '킬링타임 최고', '킬링타임', '..."
4,10097178,명불허전 선동영화 10자,0,"[-0.0144145945, 0.01716871, -0.008832245, -0.0...","{'keyword': ['명불허전 선동 영화', '명불허전 선동', '명불허전', ..."
5,9245590,결말이 마음에 든다. 소설을 먼저 읽었던 터라 결말이 마음에 든다. 세상의 모든 정...,1,"[-0.016013678, -0.053389404, -0.021212664, -0....","{'keyword': ['모든 정혜 힘내', '세상 모든', '소설 먼저 읽', '..."
6,4227156,기대했는데.. 시나리오는 ...어떻게 결말이 그렇지?..요즘 막장 드라마 영향인가..,0,"[0.002429327, -0.049576312, 0.018512795, -0.02...","{'keyword': ['시나리오 어떻 결말', '요즘 막장 드라마', '어떻', ..."
7,7313784,인기연예인 몇명 출현한다고 재미있는 영화냐 잡것들아긴급조치19호가 그렇게 만들고 망...,0,"[-0.0073289573, -0.006820913, 0.015608558, -0....","{'keyword': ['영화 냐 잡것', '인기 연예인 몇', '망하 거 못', ..."
8,3582981,좋았음!!,1,"[0.014209898, -0.02372266, -0.077045135, 0.027...","{'keyword': ['좋', '좋 음', '음'], 'score': [0.592..."
9,9906269,멜로와 스릴러를 이렇게 조합할수 있다니 ...,1,"[-0.03784318, -0.05118896, -0.009433974, -0.02...","{'keyword': ['스릴러 이렇 조합', '멜로', '조합', '있', '이렇..."


In [15]:
%%thanosql
SELECT document, label, keyword -> 'keyword' AS keywords, keyword -> 'score' AS score
FROM (
    SEARCH KEYWORD 
    USING nsmc_text_search_model
    OPTIONS (
        text_col="document",
        use_stopwords=True
        )
    AS 
    SELECT * 
    FROM nsmc_test
    LIMIT 10
)

Searching...


,document,label,keywords,score
0,꼭 한번 봐야하는 영화인 것 같네요 ㅎㅎ,1,"[꼭 번 영화, 번 영화 네요, 꼭, 네요, 영화]","[0.6515, 0.6287, 0.5034, 0.4578, 0.4226]"
1,권선징악은안드로메다로~럽라인은작가빙의된니끼한조연과~여주는남자갖고논질나쁜여자,0,"[권선징악 안드로메다 ~럽, 조연 ~여주 남자, 질 나쁘 여자, 작가 빙의, 남자 ...","[0.5268, 0.4413, 0.3591, 0.3053, 0.2873]"
2,유쾌하고 신나는 스피드 코미디.,1,"[스피드 코미디, 유쾌, 유쾌 신 나, 나 스피드, 신 나]","[0.7291, 0.6491, 0.589, 0.4306, 0.3655]"
3,정말 따뜻했던 드라마ㅠㅠ,1,"[정말 따뜻하 드라마, 정말 따뜻하, 따뜻하 드라마, 정말, 드라마]","[0.9074, 0.8146, 0.789, 0.6757, 0.4853]"
4,솔직히 배우들의 연기는 나름 괜찮았다 하지만 냉정하게 영화자체만을 놓고보면 별다른 ...,0,"[연출력 그냥 똥, 솔직히 배우 연기, 나름 괜찮 하지만, 냉정 영화 자체, 특색 ...","[0.5622, 0.4827, 0.3669, 0.3594, 0.3401]"
5,느와르에 멜로에 신파극까지······. 짬뽕이 아니라 꿀꿀이죽,0,"[느와르 멜로 신파극, 신파극 짬뽕, 짬뽕 꿀꿀이죽, 멜로, 느와르]","[0.5486, 0.4797, 0.3785, 0.3647, 0.3311]"
6,다시 보고 싶네요~,1,"[다시 싶 네요, 다시 싶, 네요, 다시, 싶]","[0.8076, 0.6742, 0.5991, 0.5987, 0.4765]"
7,짱이야... 진짜 극장을 나오는데 멋진 소설을 한권 읽은 듯한 기분을 느낌. 재미는...,1,"[넘치 영화 꼭, 짱 야 진짜, 감동 철철, 재미 물론 감동, 소설 권 읽]","[0.565, 0.5486, 0.4997, 0.4675, 0.2737]"
8,유일하게 아들과 같이볼수있는 만화영화,1,"[있 만화 영화, 같이 있 만화, 유일 아들 같이, 아들 같이 있, 유일]","[0.6405, 0.6172, 0.5625, 0.5438, 0.4235]"
9,남자라면 봐라 꼭봐라 두번봐라 세번봐라 계속봐라,1,"[번 어라 계속, 계속, 두 번, 꼭, 남자 라면 보]","[0.4835, 0.4831, 0.4082, 0.3831, 0.2687]"


In [16]:
%%thanosql 
SELECT * FROM (
    SELECT document, label, json_array_elements(keyword -> 'keyword') AS keywords, (json_array_elements(keyword -> 'score'))::text::float AS score
        FROM (
            SEARCH KEYWORD 
            USING nsmc_text_search_model
            OPTIONS (
                text_col="document",
                use_stopwords=True
                )
            AS 
            SELECT * 
            FROM nsmc_test
            LIMIT 10
        )
    ) 
WHERE score > 0.5

Searching...


,document,label,keywords,score
0,꼭 한번 봐야하는 영화인 것 같네요 ㅎㅎ,1,꼭 번 영화,0.6515
1,꼭 한번 봐야하는 영화인 것 같네요 ㅎㅎ,1,번 영화 네요,0.6287
2,꼭 한번 봐야하는 영화인 것 같네요 ㅎㅎ,1,꼭,0.5034
3,권선징악은안드로메다로~럽라인은작가빙의된니끼한조연과~여주는남자갖고논질나쁜여자,0,권선징악 안드로메다 ~럽,0.5268
4,유쾌하고 신나는 스피드 코미디.,1,스피드 코미디,0.7291
5,유쾌하고 신나는 스피드 코미디.,1,유쾌,0.6491
6,유쾌하고 신나는 스피드 코미디.,1,유쾌 신 나,0.5890
7,정말 따뜻했던 드라마ㅠㅠ,1,정말 따뜻하 드라마,0.9074
8,정말 따뜻했던 드라마ㅠㅠ,1,정말 따뜻하,0.8146
9,정말 따뜻했던 드라마ㅠㅠ,1,따뜻하 드라마,0.7890


<div class="admonition note">
    <h4 class="admonition-title">쿼리 세부 정보</h4>
    <ul>
        <li>"<strong>SEARCH KEYWORD [images|audio|videos|texts|keywords]</strong>" 쿼리 구문은 검색하고자 하는 이미지|오디오|비디오|텍스트|키워드 데이터를 정의합니다.</li>
        <li>"<strong>USING</strong>"은 텍스트 수치화에 사용할 모델을 정의합니다.</li>
        <li>"<strong>OPTIONS</strong>" 쿼리 구문을 통해 텍스트 수치화 시 필요한 변수들을 정의합니다.
            <ul>
                <li>"lang" : en, ko</li>
                <li>"text_col" : 텍스트가 있는 열 이름</li>
                <li>"ngram_range" : 키워드의 최소 단어 수와 최대 단어수. 예) [1, 3]. 대부분의 상황에서 키워드는 최대 단어 수에 맞춰 추출됩니다 (list[int, int], DEFAULT : [1, 2])</li>
                <li>"top_n" : 추출할 키워드의 수, 유사도가 높은 순서대로 (int, DEFAULT : 5)</li>
                <li>"diversity" : 추출될 키워드의 다양성. 높을 수록 기존에 추출된 키워드와 유사한 키워드는 다시 추출되지 않습니다. 0 <= diversity <= 1 (float, DEFAULT : 0.5)</li>
                <li>"use_stopwords" : 큰 의미가 없는 단어(불용어)를 제외할 지 여부 (True|False, DEFAULT : True)</li>
                <li>"threshold" : 추출할 키워드의 유사도 수치의 최소값 (float, DEFAULT : 0.0)</li>
            </ul>
        <li>"<strong>AS</strong>" 쿼리 구문은 검색에 사용할 임베딩 테이블을 정의합니다. <code>nsmc_test</code> 테이블을 사용합니다.</li>
    </ul>
</div>

## __5. 두 방법 결합하여 사용하기__

In [20]:
%%thanosql
SEARCH KEYWORD
USING nsmc_text_search_model
OPTIONS (
    text_col="document",
    ngram_range=[1, 3],
    use_stopwords=True
    )
AS (
    SELECT document, label, search_result as score
    FROM (
        SEARCH TEXT text="가볍게 볼 수 있는 코미디 영화"
        USING nsmc_text_search_model
        OPTIONS(emb_col = 'convert_result')
        AS 
        SELECT * 
        FROM nsmc_test
        )
    ORDER BY score DESC 
    LIMIT 10
)

Searching...
Searching...


,document,label,score,keyword
0,인상 깊은 영화다~,1,0.651514,"{'keyword': ['인상 깊 영화', '깊 영화', '영화', '인상', '깊..."
1,"""진부한영화 강북고 서한길 남김..""""""",0,0.649277,"{'keyword': ['한길 남기 ᆷ', '남기', '강북 서 한길', '진부하 ..."
2,"""대박이다 이시대 최고의 영화! """"뭐야, 론머맨이잖아! 론머맨~~치지지지""""""",1,0.648993,"{'keyword': ['최고 영화 뭐', '시대 최고', '론머맨 잖아 론머맨',..."
3,"""전체관람가가 아니라 """"미취학아동전용""""이네요""",0,0.648626,"{'keyword': ['아동 전용 네요', '아동', '전체', '가 미 취학',..."
4,"""""""세상은 친절한 곳이 아니에요""""""",1,0.636068,"{'keyword': ['세상 친절 곳', '세상', '에요', '곳', '친절']..."
5,"""""""다녀왔습니다""""가 제일 무서웠음.""",0,0.633658,"{'keyword': ['습니다', '다녀오', '일', '음', '일 무섭'], ..."
6,감동적인 영화,1,0.620944,"{'keyword': ['감동 영화', '감동', '영화'], 'score': [0..."
7,"""""""관심받고 싶었습니다. 여러분을 낚은거 죄송"""" 맞제?""",0,0.620719,"{'keyword': ['싶 습니다 여러분', '관심 받', '맞 제', '여러분 ..."
8,당시에 '천사의 미소'로 방송되었던 재밌게 봤었다.,1,0.616029,"{'keyword': ['당시 천사 미소', '방송 재밌 었었', '미소 방송 재밌..."
9,소소한 재미로 매력 있는 영화! 예기치않게 빵빵터지는 영화~!,1,0.616009,"{'keyword': ['재미 매력 영화', '영화 예기 빵빵', '예기 빵빵 터지..."


In [21]:
%%thanosql
SELECT document, label, keyword -> 'keyword' AS keywords, keyword -> 'score' AS score
FROM ( 
    SEARCH KEYWORD
    USING nsmc_text_search_model
    OPTIONS (
        text_col="document",
        ngram_range=[1, 3],
        use_stopwords=True
        )
    AS (
        SELECT document, label, search_result as score
        FROM (
            SEARCH TEXT text="가볍게 볼 수 있는 코미디 영화"
            USING nsmc_text_search_model
            OPTIONS(emb_col = 'convert_result')
            AS 
            SELECT * 
            FROM nsmc_test
            )
        ORDER BY score DESC 
        LIMIT 10
    )
)

Searching...
Searching...


,document,label,keywords,score
0,인상 깊은 영화다~,1,"[인상 깊 영화, 깊 영화, 영화, 인상, 깊]","[0.8289, 0.6993, 0.5736, 0.5725, 0.5506]"
1,"""진부한영화 강북고 서한길 남김..""""""",0,"[한길 남기 ᆷ, 남기, 강북 서 한길, 진부하 영화 강북, 서]","[0.4233, 0.3914, 0.3746, 0.3175, 0.3141]"
2,"""대박이다 이시대 최고의 영화! """"뭐야, 론머맨이잖아! 론머맨~~치지지지""""""",1,"[최고 영화 뭐, 시대 최고, 론머맨 잖아 론머맨, 박, 치지지지]","[0.3485, 0.3365, 0.2687, 0.2372, 0.1448]"
3,"""전체관람가가 아니라 """"미취학아동전용""""이네요""",0,"[아동 전용 네요, 아동, 전체, 가 미 취학, 관람 가 가]","[0.3698, 0.3359, 0.3168, 0.2547, 0.1344]"
4,"""""""세상은 친절한 곳이 아니에요""""""",1,"[세상 친절 곳, 세상, 에요, 곳, 친절]","[0.2729, 0.2707, 0.2492, 0.2406, 0.2111]"
5,"""""""다녀왔습니다""""가 제일 무서웠음.""",0,"[습니다, 다녀오, 일, 음, 일 무섭]","[0.3643, 0.2608, 0.2241, 0.1953, 0.188]"
6,감동적인 영화,1,"[감동 영화, 감동, 영화]","[0.9238, 0.7883, 0.6925]"
7,"""""""관심받고 싶었습니다. 여러분을 낚은거 죄송"""" 맞제?""",0,"[싶 습니다 여러분, 관심 받, 맞 제, 여러분 낚 거, 거 죄 송]","[0.4304, 0.2753, 0.2633, 0.2588, 0.1219]"
8,당시에 '천사의 미소'로 방송되었던 재밌게 봤었다.,1,"[당시 천사 미소, 방송 재밌 었었, 미소 방송 재밌, 당시, 방송]","[0.5619, 0.5586, 0.5315, 0.4045, 0.3622]"
9,소소한 재미로 매력 있는 영화! 예기치않게 빵빵터지는 영화~!,1,"[재미 매력 영화, 영화 예기 빵빵, 예기 빵빵 터지, 터지 영화, 소소]","[0.6433, 0.5984, 0.4859, 0.4783, 0.3455]"


In [24]:
%%thanosql
SELECT * 
FROM 
    (SELECT document, label, json_array_elements(keyword -> 'keyword') AS keywords, (json_array_elements(keyword -> 'score'))::text::float AS score
    FROM (
        SEARCH KEYWORD
        USING nsmc_text_search_model
        OPTIONS (
            text_col="document",
            ngram_range=[1, 3],
            use_stopwords=True
            )
        AS (
            SELECT document, label, search_result as score
            FROM (
                SEARCH TEXT text="최고의 액션 영화"
                USING nsmc_text_search_model
                OPTIONS(emb_col = 'convert_result')
                AS 
                SELECT * 
                FROM nsmc_test
                WHERE document LIKE '%%판타지%%'
                )
            ORDER BY score DESC
            LIMIT 10
        )
    )
)
WHERE score > 0.3

Searching...
Searching...


,document,label,keywords,score
0,정말 재밌다.이런게 바로 판타지모험영화의 현실이다.,1,판타지 모험 영화,0.6481
1,정말 재밌다.이런게 바로 판타지모험영화의 현실이다.,1,바로 판타지 모험,0.5873
2,정말 재밌다.이런게 바로 판타지모험영화의 현실이다.,1,영화 현실,0.5638
3,정말 재밌다.이런게 바로 판타지모험영화의 현실이다.,1,이런 바로 판타지,0.5630
4,정말 재밌다.이런게 바로 판타지모험영화의 현실이다.,1,정말 재밌 이런,0.5387
5,30년 전의 퓨전 판타지라니..그냥 찬양해야됨,1,라니 그냥 찬양,0.6097
6,30년 전의 퓨전 판타지라니..그냥 찬양해야됨,1,그냥 찬양 ᆷ,0.5967
7,30년 전의 퓨전 판타지라니..그냥 찬양해야됨,1,퓨전 판타지,0.5534
8,30년 전의 퓨전 판타지라니..그냥 찬양해야됨,1,찬양 ᆷ,0.3952
9,1700만 1700만 해되서 봤다가 욕만하며 참고 참으며 보다 결국 사분의 일은 스...,0,삼류 판타지 아류작,0.3783


<div class="admonition note">
    <h4 class="admonition-title">쿼리 세부 정보</h4>
    <ul>
        <li>"<strong>SEARCH TEXT [images|audio|videos|texts|keywords]</strong>" 쿼리 구문은 검색하고자 하는 이미지|오디오|비디오|텍스트|키워드 데이터를 정의합니다.</li>
        <li>"<strong>SEARCH KEYWORD [images|audio|videos|texts|keywords]</strong>" 쿼리 구문은 검색하고자 하는 이미지|오디오|비디오|텍스트|키워드 데이터를 정의합니다.</li>
        <li>"<strong>USING</strong>"은 텍스트 수치화에 사용할 모델을 정의합니다.</li>
        <li>"<strong>AS</strong>" 쿼리 구문은 검색에 사용할 임베딩 테이블을 정의합니다. <code>nsmc_test</code> 테이블을 사용합니다.</li>
    </ul>
</div>

## __6. 튜토리얼을 마치며__

이번 튜토리얼에서는 `nsmc` 영화 리뷰 데이터를 사용하여 텍스트 수치화와 수치화 결과를 바탕으로 한 유사 텍스트 검색, 키워드 추출을 진행해 보았습니다. 이번 튜토리얼에서는 모델의 수치화 성능보다는 작동 위주의 설명으로 진행하였습니다. 나만의 텍스트 수치화모델을 만들어 다양한 형태의 비정형 데이터 세트에 검색 기능을 추가하고 Auto-ML 기법을 이용한 나만의 모델을 배포할 수 있습니다.
<br>
다음 단계에서 텍스트 수치화 모델의 다양한 "__OPTIONS__" 쿼리 구문과 학습 방법을 더욱 심도있게 다뤄봅니다. 나만의 정확한 텍스트 수치화 모델 구축방법에 대해 더욱 자세히 알고 싶다면 다음 튜토리얼들을 진행 해보세요.

* [나만의 데이터 업로드하기](https://docs.thanosql.ai/getting_started/data_upload/)
* [중급 유사 텍스트 검색 모델 만들기]

<div class="admonition tip">
    <h4 class="admonition-title">나만의 서비스를 위한 모델 배포 관련 문의</h4>
    <p>ThanoSQL을 활용해 나만의 모델을 만들거나, 나의 서비스에 적용하는데 어려움이 있다면 언제든 아래로 문의주세요😊</p>
    <p>유사 텍스트 검색 모델 구축 관련 문의: contact@smartmind.team</p>
</div>